# Dataset Statistics for SynthDet
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [Perception package](https://github.com/Unity-Technologies/com.unity.perception) and visualize dataset statistics. It includes statistics and visualizations of the outputs built into the Perception package and should give a good idea of how to use datasetinsights to visualize custom annotations and metrics.

## Setup dataset
If the dataset was generated locally, replace `<GUID>` below with the name of the dataset folder, e.g. `f3763556-355f-4303-9acd-32334fda51aa`. Otherwise, see the instructions below for downloading datasets from Unity Simulation.

In [ ]:
data_root = "</data/<GUID>"

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `access_token` in the following cell. The `access_token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [ ]:
from datasetinsights.io.downloader import UnitySimulationDownloader

#run execution id:
# run_execution_id = "xxx"
# #access_token:
# access_token = "xxx"
# #annotation definition id:
# annotation_definition_id = "6716c783-1c0e-44ae-b1b5-7f068454b66e"
# #unity project id
# project_id = "xxx"
# source_uri = f"usim://{project_id}/{run_execution_id}"

# downloader = UnitySimulationDownloader(access_token=access_token)

Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation.


In [ ]:
# downloader.download(source_uri=source_uri, output=data_root)

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.data.simulation`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [ ]:
from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions
ann_def = AnnotationDefinitions(data_root)
ann_def.table

In [ ]:
metric_def = MetricDefinitions(data_root)
metric_def.table

## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.data.datasets.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [ ]:
from datasetinsights.stats.statistics import RenderedObjectInfo
from datasetinsights.datasets.unity_perception.exceptions import DefinitionIDError
from datasetinsights.stats import bar_plot, histogram_plot, rotation_plot

max_samples = 10000          # maximum number of sample points used in histogram plots

rendered_object_info_definition_id = "659c6e36-f9f8-4dd6-9651-4a80e51eabc4"
roinfo = None
try:
    roinfo = RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

### Descriptive Statistics

In [ ]:
print(roinfo.num_captures())
roinfo.raw_table.head(3)

### Total Object Count

In [ ]:
total_count = roinfo.total_counts()
display(total_count)

display(bar_plot(
    total_count, 
    x="label_id", 
    y="count", 
    x_title="Label Name",
    y_title="Count",
    title="Total Object Count in Dataset",
    hover_name="label_name"
))

### Per Capture Object Count

In [ ]:
per_capture_count = roinfo.per_capture_counts()
display(per_capture_count.head(10))

In [ ]:
display(histogram_plot(
    per_capture_count, 
    x="count",  
    x_title="Object Counts Per Capture",
    y_title="Frequency",
    title="Distribution of Object Counts Per Capture",
    max_samples=max_samples
))

### Object Visible Pixels

In [ ]:
display(histogram_plot(
    roinfo.raw_table, 
    x="visible_pixels",  
    x_title="Visible Pixels Per Object",
    y_title="Frequency",
    title="Distribution of Visible Pixels Per Object",
    max_samples=max_samples
))

## SynthDet Statistics
Metrics specific to the simulation can be loaded using `datasetinsights.datasets.unity_perception.Metrics`. 

### Foreground placement info

In [ ]:
from datasetinsights.datasets.unity_perception import Metrics
import pandas as pd

metrics = Metrics(data_root=data_root)
foreground_placement_info_definition_id = "061e08cc-4428-4926-9933-a6732524b52b"

def read_foreground_placement_info(metrics=None, name=None, columns=None):
    filtered_metrics = metrics.filter_metrics(foreground_placement_info_definition_id)
    combined = pd.DataFrame(filtered_metrics[name].to_list(), columns=columns)
    
    return combined

In [ ]:
rot_columns = ("x_rot", "y_rot", "z_rot")
orientation_info = read_foreground_placement_info(metrics=metrics, name="rotation", columns=rot_columns)
orientation_info.head(5)

In [ ]:
rotation_plot(
    orientation_info,
    x="x_rot",
    y="y_rot",
    z="z_rot",
    title="Object orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    orientation_info, 
    x="x_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    orientation_info, 
    x="y_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along Y direction",
    max_samples=max_samples
)

### Foreground Scale Info

In [ ]:
scale_columns = ("x_scale", "y_scale", "z_scale")
scale_info = read_foreground_placement_info(metrics=metrics, name="scale", columns=scale_columns)
scale_info.head(5)

In [ ]:
histogram_plot(
    scale_info, 
    x="x_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Scale along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    scale_info, 
    x="y_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Scale along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    scale_info, 
    x="z_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Object Scale along Z direction",
    max_samples=max_samples
)

### Foreground Position Info

In [ ]:
pos_columns = ("x_pos", "y_pos", "z_pos")
pos_info = read_foreground_placement_info(metrics=metrics, name="position", columns=pos_columns)
pos_info.head(5)

In [ ]:
histogram_plot(
    pos_info, 
    x="x_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    pos_info, 
    x="y_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    pos_info, 
    x="z_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along Z direction",
    max_samples=max_samples
)

### Lighting info 
There are 4 lights present in the SynthDet project. Statistics for each light are presented below.

In [ ]:
lighting_info_definition_id = "939248ee-668a-4e98-8e79-e7909f034a47"
x_y_columns = ["x_rotation", "y_rotation"]
color_columns = ["color.r", "color.g", "color.b", "color.a"]
other_columns = ["intensity", "enabled", "lightName"]

def read_lighting_info(metrics, light_name=None):
    filtered_metrics = metrics.filter_metrics(lighting_info_definition_id)
    filtered_metrics = filtered_metrics.reset_index()
    colors = pd.json_normalize(filtered_metrics["color"])
    colors.columns = color_columns
    
    combined = pd.concat([filtered_metrics[x_y_columns], filtered_metrics[other_columns], colors], axis=1, join="inner")
    filter_combined = combined[combined["lightName"]==light_name]
    return filter_combined

#### Directional Light_Background_1_Intermittent

In [ ]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_Background_1_Intermittent")
lighting_info.head(5)

In [ ]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="x_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="y_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_0

In [ ]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_0")
lighting_info.head(5)

In [ ]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="x_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="y_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_1

In [ ]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_1")
lighting_info.head(5)

In [ ]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="x_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="y_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_2

In [ ]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_2")
lighting_info.head(5)

In [ ]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="x_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along X direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="y_rotation",  
    x_title="Lighting Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Lighting Rotations along Y direction",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

### Camera Info

In [ ]:
lighting_info_definition_id = "a4b2253c-0eb2-4a90-9b20-6e77f1d13286"
camear_columns = ["saturation", "contrast", "gaussianDofStart", "gaussianDofEnd", "gaussianDofRadius"]

def read_camera_info(metrics):
    filtered_metrics = metrics.filter_metrics(lighting_info_definition_id)

    return filtered_metrics[camear_columns]

In [ ]:
camera_info = read_camera_info(metrics)
camera_info.head(5)

In [ ]:
histogram_plot(
    camera_info, 
    x="saturation",  
    x_title="Camera Saturation",
    y_title="Frequency",
    title="Distribution of Camera Saturation",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    camera_info, 
    x="contrast",  
    x_title="Camera Contrast",
    y_title="Frequency",
    title="Distribution of Camera Contrast",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    camera_info, 
    x="gaussianDofStart",  
    x_title="Camera Gaussian Depth of Field Start",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field Start",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    camera_info, 
    x="gaussianDofEnd",  
    x_title="Camera Gaussian Depth of Field End",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field End",
    max_samples=max_samples
)

In [ ]:
histogram_plot(
    camera_info, 
    x="gaussianDofRadius",  
    x_title="Camera Gaussian Depth of Field Radius",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field Radius",
    max_samples=max_samples
)

## Annotation Visualization
In the following sections we show how to load annotations from the Captures object and visualize them. Similar code can be used to consume annotations for model training or visualize and train on custom annotations.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [ ]:
# downloader.download(source_uri=source_uri, output=data_root, include_binary=True)

### Load captures

In [ ]:
from datasetinsights.datasets.unity_perception.captures import Captures
cap = Captures(data_root)
cap.captures.head(3)

### Bounding Boxes
In this section we render 2d bounding boxes on top of the captured images.

In [ ]:
import os

from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image

from datasetinsights.stats.visualization.plots import plot_bboxes
from datasetinsights.datasets.synthetic import SynDetection2D,read_bounding_box_2d

bounding_box_definition_id = "c31620e3-55ff-4af6-ae86-884aa0daa9b2"
dataset = SynDetection2D(data_path=data_root, def_id=bounding_box_definition_id)
def draw_bounding_boxes(index):
    image, bboxes = dataset[index]
    return plot_bboxes(image, bboxes, dataset.label_mappings)

In [ ]:
from ipywidgets import interact

# pick an index and visualize
interact(draw_bounding_boxes, index=list(range(len(dataset))))

### 3D Ground Truth Bounding Boxes
In this section we render 3d ground truth bounding boxes on top of the captured images.

In [ ]:
import os
import numpy as np

from ipywidgets import interact
from PIL import Image
from datasetinsights.stats.visualization.plots import plot_bboxes3d
from datasetinsights.datasets.synthetic import read_bounding_box_3d

bounding_box_3d_defintion_id = "0bfbe00d-00fa-4555-88d1-471b58449f5c"
def draw_bounding_boxes3d(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    sensor = box_captures.loc[index, "sensor"]

    if 'camera_intrinsic' in sensor:
        projection = np.array(sensor["camera_intrinsic"])
    else:
        projection = np.array([[1,0,0],[0,1,0],[0,0,1]])

    image = Image.open(filename)
    boxes = read_bounding_box_3d(annotations)
    img_with_boxes = plot_bboxes3d(image, boxes, projection)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    box_captures = cap.filter(def_id=bounding_box_3d_defintion_id)
    interact(draw_bounding_boxes3d, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

## Semantic Segmentation
In this section we render the semantic segmentation images on top of the captured images.

In [ ]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")

## Instance Segmentation
In this section we render the instance segmentation images on top of the captured images. Image IDs are mapped to an RGBA color value, below the image we include a preview of the mapping between colors and IDs.

In [ ]:
def instance_sorter(instance):
    return instance["instance_id"]

def draw_with_instance_segmentation(index, opacity):
    filename = os.path.join(data_root, inst_caps.loc[index, "filename"])
    seg_filename = os.path.join(data_root, inst_caps.loc[index, "annotation.filename"])

    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)

    anns = inst_caps.loc[index, "annotation.values"].copy()
    anns.sort(key=instance_sorter)

    count = min(5, len(anns))
    print("First {} ID entries:".format(count))

    for i in range(count):
        color = anns[i].get("color")
        print ("{} => Color({:>3}, {:>3}, {:>3})".format(anns[i].get("instance_id"), color.get("r"), color.get("g"), color.get("b")))

try:
    inst_seg_def_id = "1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf"
    inst_caps = cap.filter(def_id=inst_seg_def_id)
    interact(draw_with_instance_segmentation, index=(0, inst_caps.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No instance segmentation images found")
